In [2]:
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


# Visuals on Government Assistance across the US between different categories

- https://api.census.gov/data/2020/absmcb/variables.html

For this api call, we look at government assistance for companies in 2020 in the US and how much money they recieved. To clean the dataset, I took out labels that included all firms, as we were not interested in that. I also filtered out the labels that included companies that did not report thier assistance or did not claim any.

In [4]:
module_business_char2 = pd.read_csv('data/module_business_char2.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/module_business_char2.csv'

In [26]:
module_business_char_clean = module_business_char2[(module_business_char2['BUSCHAR_LABEL'] != 'All firms') & (module_business_char2['BUSCHAR'] != 'F04ATR') & (module_business_char2['BUSCHAR'] != 'F04AXX')]


## Build Dataframe on sex

To look at the companies by sex, I filtered the previously cleaned dataset to only show female, male and equally male and female owned businesses. Also made sure it did not break down the companies any further. The dataframe wanted to also group them by different race groups bu twe made sure it did not.

In [11]:
module_busineess_male_female =  module_business_char_clean[((module_business_char_clean['SEX_LABEL'] == 'Female') | (module_business_char_clean['SEX_LABEL'] == 'Male') | (module_business_char_clean['SEX_LABEL'] == 'Equally male/female'))& (module_business_char_clean['RACE_GROUP_LABEL'] == 'Total') & (module_business_char_clean['VET_GROUP_LABEL'] == 'Total')& (module_business_char_clean['ETH_GROUP_LABEL'] == 'Total')]

In [13]:
fig = px.bar(module_busineess_male_female, pattern_shape=module_busineess_male_female['SEX_LABEL'].astype(str), pattern_shape_sequence=[".", "+", '/'], color = module_busineess_male_female['SEX_LABEL'].astype(str), barmode='group', x=module_busineess_male_female['BUSCHAR_LABEL'].astype(str), y=module_busineess_male_female['FIRMPDEMP'].astype(float),
labels=dict(x='Amount of money recieved', color = 'Sex', pattern_shape='Sex', y='Number of firms'), title='Government Assistance for Majority Male Vs Female Owned Businesses')
# fig.update_xaxes(showticklabels=False)
fig.update_xaxes(minor_tickmode="array")
fig.update_xaxes(tickvals=[0,1,2,3,4,5,6,7,8,9])
fig.update_xaxes(ticktext=['Less than $5k', '$5k-$9,999', '$10k-24,999', '$25k-$49,999', '$50k-$99,999', '$100k-$249,999','250k-$499,999','$500k-$999,999','$1,000,000 or more'])
fig.show()

## Assistance by Race

Looking at the assistance by race groups went a lot like male vs female. Instead we filtered the dataframe to be in the race groups and not devided by other catergories such as sex.

In [15]:
module_busineess_by_race =  module_business_char_clean[(module_business_char_clean['SEX_LABEL'] == 'Total')& ((module_business_char_clean['RACE_GROUP_LABEL'] == 'Asian') | (module_business_char_clean['RACE_GROUP_LABEL'] == 'Black or African American') | (module_business_char_clean['RACE_GROUP_LABEL'] == 'American Indian and Alaska Native') | (module_business_char_clean['RACE_GROUP_LABEL'] == 'Native Hawaiian and Other Pacific Islander') | (module_business_char_clean['RACE_GROUP_LABEL'] == 'Equally minority/nonminority') ) & (module_business_char_clean['VET_GROUP_LABEL'] == 'Total')& (module_business_char_clean['ETH_GROUP_LABEL'] == 'Total')]

In [16]:
module_busineess_by_race_white =  module_business_char_clean[(module_business_char_clean['SEX_LABEL'] == 'Total')& ((module_business_char_clean['RACE_GROUP_LABEL'] == 'White')) & (module_business_char_clean['VET_GROUP_LABEL'] == 'Total')& (module_business_char_clean['ETH_GROUP_LABEL'] == 'Total')]

In [18]:
fig = px.bar(module_busineess_by_race, pattern_shape=module_busineess_by_race['RACE_GROUP_LABEL'].astype(str), pattern_shape_sequence=[".", "+", '/'], color = module_busineess_by_race['RACE_GROUP_LABEL'].astype(str), barmode='stack', x=module_busineess_by_race['BUSCHAR_LABEL'].astype(str), y=module_busineess_by_race['FIRMPDEMP'].astype(float),
labels=dict(x='Amount of money recieved', color = 'Race', pattern_shape='Race', y='Number of firms'), title='Government Assistance by Race Group')
# fig.update_xaxes(showticklabels=False)
fig.update_xaxes(minor_tickmode="array")
fig.update_xaxes(tickvals=[0,1,2,3,4,5,6,7,8,9])
fig.update_xaxes(ticktext=['Less than $5k', '$5k-$9,999', '$10k-24,999', '$25k-$49,999', '$50k-$99,999', '$100k-$249,999','250k-$499,999','$500k-$999,999','$1,000,000 or more'])
fig.show()
fig = px.bar(module_busineess_by_race_white, pattern_shape=module_busineess_by_race_white['RACE_GROUP_LABEL'].astype(str), pattern_shape_sequence=[".", "+", '/'], color = module_busineess_by_race_white['RACE_GROUP_LABEL'].astype(str), barmode='stack', x=module_busineess_by_race_white['BUSCHAR_LABEL'].astype(str), y=module_busineess_by_race_white['FIRMPDEMP'].astype(float),
labels=dict(x='Amount of money recieved', color = 'Race', pattern_shape='Race', y='Number of firms'), title='Government Assistance for White Owned Business')
# fig.update_xaxes(showticklabels=False)
fig.update_xaxes(minor_tickmode="array")
fig.update_xaxes(tickvals=[0,1,2,3,4,5,6,7,8,9])
fig.update_xaxes(ticktext=['Less than $5k', '$5k-$9,999', '$10k-24,999', '$25k-$49,999', '$50k-$99,999', '$100k-$249,999','250k-$499,999','$500k-$999,999','$1,000,000 or more'])
fig.show()

## Requested Assitance

For this we changed the api call to see how many compnaies requested how much money. We then cleaned it like the previous call and focused on the race groups while ignoring catergories such as sex, vet status and etc.

In [19]:
url7 = 'https://api.census.gov/data/2020/absmcb?get=GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,QDESC,QDESC_LABEL,BUSCHAR,BUSCHAR_LABEL,YEAR,FIRMPDEMP&for=us:*&&QDESC_LABEL=GOVFAREQ&NAICS2017=00&key={}'.format(api_key)
response7 = requests.request('GET',url7)
response7

<Response [200]>

In [20]:
module_business_char3 = json_to_dateframe(response7)

In [22]:
module_business_requested_help = module_business_char3[(module_business_char3['BUSCHAR_LABEL'] != 'All firms') & (module_business_char3['BUSCHAR'] != 'F03ATR') & (module_business_char3['BUSCHAR'] != 'F03AXX')]


In [23]:
module_busineess_by_race_requested =  module_business_requested_help[(module_business_requested_help['SEX_LABEL'] == 'Total')& ((module_business_requested_help['RACE_GROUP_LABEL'] == 'Asian') | (module_business_requested_help['RACE_GROUP_LABEL'] == 'Black or African American') | (module_business_requested_help['RACE_GROUP_LABEL'] == 'American Indian and Alaska Native') | (module_business_requested_help['RACE_GROUP_LABEL'] == 'Native Hawaiian and Other Pacific Islander') | (module_business_requested_help['RACE_GROUP_LABEL'] == 'Equally minority/nonminority') ) & (module_business_requested_help['VET_GROUP_LABEL'] == 'Total')& (module_business_requested_help['ETH_GROUP_LABEL'] == 'Total')]


In [24]:
module_busineess_by_race_white_requested =  module_business_requested_help[(module_business_requested_help['SEX_LABEL'] == 'Total')& ((module_business_requested_help['RACE_GROUP_LABEL'] == 'White')) & (module_business_requested_help['VET_GROUP_LABEL'] == 'Total')& (module_business_requested_help['ETH_GROUP_LABEL'] == 'Total')]

In [25]:
fig = px.bar(module_busineess_by_race_requested, pattern_shape=module_busineess_by_race_requested['RACE_GROUP_LABEL'].astype(str), pattern_shape_sequence=[".", "+", '/'], color = module_busineess_by_race_requested['RACE_GROUP_LABEL'].astype(str), barmode='stack', x=module_busineess_by_race_requested['BUSCHAR_LABEL'].astype(str), y=module_busineess_by_race_requested['FIRMPDEMP'].astype(float),
labels=dict(x='Amount of money requested', color = 'Race', pattern_shape='Race', y='Number of firms'), title='Government Assistance Requested by race group')
# fig.update_xaxes(showticklabels=False)
fig.update_xaxes(minor_tickmode="array")
fig.update_xaxes(tickvals=[0,1,2,3,4,5,6,7,8,9])
fig.update_xaxes(ticktext=['Less than $5k', '$5k-$9,999', '$10k-24,999', '$25k-$49,999', '$50k-$99,999', '$100k-$249,999','250k-$499,999','$500k-$999,999','$1,000,000 or more'])
fig.show()
fig = px.bar(module_busineess_by_race_white_requested, pattern_shape=module_busineess_by_race_white_requested['RACE_GROUP_LABEL'].astype(str), pattern_shape_sequence=[".", "+", '/'], color = module_busineess_by_race_white_requested['RACE_GROUP_LABEL'].astype(str), barmode='stack', x=module_busineess_by_race_white_requested['BUSCHAR_LABEL'].astype(str), y=module_busineess_by_race_white_requested['FIRMPDEMP'].astype(float),
labels=dict(x='Amount of money recieved', color = 'Race', pattern_shape='Race', y='Number of firms'), title='Government Assistance Requested for white business onwers')
# fig.update_xaxes(showticklabels=False)
fig.update_xaxes(minor_tickmode="array")
fig.update_xaxes(tickvals=[0,1,2,3,4,5,6,7,8,9])
fig.update_xaxes(ticktext=['Less than $5k', '$5k-$9,999', '$10k-24,999', '$25k-$49,999', '$50k-$99,999', '$100k-$249,999','250k-$499,999','$500k-$999,999','$1,000,000 or more'])
fig.show()